In [1]:
import numpy as np

path_prefix = "./all/data/"
word_to_index = {} # map word to index
g_word_list = [] # sorted by frequence low to high
total_word_dict = {}
skip_words = set(['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'as', 'an', 'and', 'are', 'at', 'by', 'down', 'for', 'from', 'front', 'has', 'in', 'is', 'it', 'near', 'next', 'of', 'on', 'out', 'over', 'several', 'sits', 'some', 'that', 'the', 'there', 'through', 'to', 'top', 'up', 'while', 'with'])
train_des = [] #0/1/2 array of word
test_des = [] #0/1/2 array of word
train_feature_list = [] # list of feature_array by index
test_feature_list = []
all_predict_test_des = []
train_g_category_tag = []
train_g_item_tag = []
test_g_category_tag = []
test_g_item_tag = []
MAX_INPUT = 100
MAX_OUTPUT = 20
COMPONENTS = 400 # need to run again
PCA_NUMBER = 2000 # need to run again
ITER = 500

In [2]:
def read_file_to_string(file_name, keep_colon):
    f = open(file_name, "r")
    content = f.read()
    f.close()
    content = content.lower()
    content = content.replace(",", " ").replace(".", " ").replace("'", " ").replace("\"", " ")
    if (keep_colon == False):
        content = content.replace(":", " ")
    return content

def read_file_to_total_word_dict(file_name, keep_colon):
    word_list = read_file_to_string(file_name, keep_colon).split()
    for name in word_list:
        if name in skip_words:
            continue
        if name in total_word_dict:
            total_word_dict[name] += 1
            continue
        else:
            total_word_dict[name] = 1
            continue

def print_word_in_array(a):
    ret = ""
    for i in range(len(a)):
        if a[i] != 0:
            ret = ret + str(a[i]) + g_word_list[i] + " "
    print(ret)

In [3]:
total_word_dict = {}
for i in range(MAX_INPUT):
    read_file_to_total_word_dict(path_prefix+"descriptions_train/"+str(i)+".txt", False)
    read_file_to_total_word_dict(path_prefix+"tags_train/"+str(i)+".txt", True)

In [4]:
import functools
def cmp(a, b):
    a_num = abs(float(a.split("___")[0]))
    b_num = abs(float(b.split("___")[0]))
    if (a_num < b_num):
        return -1
    return 1

In [5]:
# load g_word_list
g_word_list = []
word_to_index = {}
tmp_list = []
#TODO: maybe try filter on number
for word in total_word_dict:
    if total_word_dict[word] <= 0:
        continue
    tmp_list.append(str(total_word_dict[word])+"___"+word)

tmp_list = sorted(tmp_list, key=functools.cmp_to_key(cmp))

for i in range(len(tmp_list)):
    word = tmp_list[i].split("___")[1]
    g_word_list.append(word)
    word_to_index[word] = i

In [6]:
# read to tags
train_g_category_tag = [0] * MAX_INPUT
train_g_item_tag = [0] * MAX_INPUT
test_g_category_tag = [0] * MAX_OUTPUT
test_g_item_tag = [0] * MAX_OUTPUT

#TODO: add tags in descriptions
def is_in_tag(file_number, word, train):
    if train:
        return (word in train_g_category_tag[file_number]) or (word in train_g_item_tag[file_number])
    return (word in test_g_category_tag[file_number]) or (word in test_g_item_tag[file_number])

def read_tags(file_name, train):
    lines = read_file_to_string(file_name, True).split("\n")
    index = int(file_name.split("/")[-1].replace(".txt", ""))
    current_category_set = set([])
    current_item_set = set([])
    
    for line in lines:
        cat = line.split(":")[0].strip()
        if cat in word_to_index:
            current_category_set.add(cat)

        item = ''
        if (cat.strip() != line.replace(":", "").strip()):
            item = line.split(":")[1].strip()
        if item in word_to_index:
            current_item_set.add(item)
    if (train):
        train_g_category_tag[index] = current_category_set
        train_g_item_tag[index] = current_item_set
    else:
        test_g_category_tag[index] = current_category_set
        test_g_item_tag[index] = current_item_set
    

for i in range(MAX_INPUT):
    read_tags(path_prefix+"tags_train/"+str(i)+".txt", True)
for i in range(MAX_OUTPUT):
    read_tags(path_prefix+"tags_test/"+str(i)+".txt", False)

In [7]:
# where to make the change
# todo: remove count to match test output
# lenpi ci, yaoshi you dehua, jianfen
# also try norm

# count_max_weight = 2.0
# uniqueness_max_weight = 2.0
# TODO: add words in tag_item to pic
def my_weight_func(index, count, train, file_num):
    if (count <= 0):
        return 0
#     count_apply = 0
#     if is_in_tag(file_num, g_word_list[index], train):
#         count_apply = 1
#     elif count >= 3:
#         count_apply = 1
#     return 1 + (len(g_word_list)-index)*uniqueness_max_weight/len(g_word_list) + count_apply*count_max_weight

#     if count >= 5:
#         return count
#     if is_in_tag(file_num, g_word_list[index], train):
#         return 5
    return 1

def weight_occurance_array(occurance_array, train, file_num):
    ret = [0] * len(occurance_array)
    for index in range(len(occurance_array)):
        ret[index] = my_weight_func(index, occurance_array[index], train, file_num)
    return ret

def read_des_to_list(path, file_num):
    ret = []
    train = (path.find("train") != -1)
    for i in range(file_num):
        int_array = [0] * len(g_word_list)
        temp_list = read_file_to_string(path+str(i)+".txt", False).split()
        for word in temp_list:
            if not word in word_to_index:
                continue
            int_array[word_to_index[word]] += 1
        ret.append(weight_occurance_array(int_array, train, i))
    return ret

train_des = read_des_to_list(path_prefix+"descriptions_train/", MAX_INPUT)
test_des = read_des_to_list(path_prefix+"descriptions_test/", MAX_OUTPUT)

In [8]:
# add tag to desc list

#TODO: careful with tag number
def add_tag_to_train_des():
    for sentence_index in range(MAX_INPUT):
        array_to_add = [0] * len(g_word_list)
        for tag in train_g_item_tag[sentence_index]:
            array_to_add[word_to_index[tag]] = 1
        
        ret = train_des[sentence_index].copy()
        for word_index in range(len(g_word_list)):
            ret[word_index] = max(ret[word_index], array_to_add[word_index])
        train_des[sentence_index] = ret

def add_tag_to_test_des():
    for sentence_index in range(MAX_OUTPUT):
        array_to_add = [0] * len(g_word_list)
        for tag in test_g_item_tag[sentence_index]:
            array_to_add[word_to_index[tag]] = 1
        
        ret = all_predict_test_des[sentence_index].copy()
        print(all_predict_test_des[sentence_index])
        for word_index in range(len(g_word_list)):
            ret[word_index] = max(ret[word_index], array_to_add[word_index])
        all_predict_test_des[sentence_index] = ret


In [9]:
add_tag_to_train_des()

In [10]:
# load feature list
def load_feature_into_list(filename):
    f = open(filename, "r")
    content = f.readlines()
    f.close()
    train = filename.find("train")
    for line in content:
        name_index = int(line.split(".jpg,")[0].split("/")[1])
        feature_list = line.split(".jpg,")[1].split(",")
        if train != -1:
            if name_index >= MAX_INPUT:
                continue
            train_feature_list[name_index] = feature_list
        else:
            if name_index >= MAX_OUTPUT:
                continue
            test_feature_list[name_index] = feature_list

train_feature_list = [0] * MAX_INPUT
test_feature_list = [0] * MAX_OUTPUT
# load_feature_into_list(path_prefix+"features_train/features_resnet1000_train.csv")
# load_feature_into_list(path_prefix+"features_test/features_resnet1000_test.csv")
load_feature_into_list(path_prefix+"features_train/features_resnet1000intermediate_train.csv")
load_feature_into_list(path_prefix+"features_test/features_resnet1000intermediate_test.csv")

train_feature_list = np.asarray(train_feature_list, dtype=np.float64)
test_feature_list = np.asarray(test_feature_list, dtype=np.float64)

In [11]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components = PCA_NUMBER)
# pca.fit(train_des)
# train_des_pca = pca.transform(train_des)
# test_des_pca = pca.transform(test_des)
train_des_pca = np.asarray(train_des)
test_des_pca = np.asarray(test_des)

print(train_des_pca.shape)
print(test_des_pca.shape)

# pca2 = PCA(n_components = PCA_NUMBER)
# pca2.fit(train_feature_list)
# train_feature_list = pca2.transform(train_feature_list)
# test_feature_list = pca2.transform(test_feature_list)
train_feature_list = np.asarray(train_feature_list)
test_feature_list = np.asarray(test_feature_list)

print(train_feature_list.shape)
print(test_feature_list.shape)

# from sklearn.cross_decomposition import PLSRegression
# print("Done 0")
# model = PLSRegression(COMPONENTS, max_iter=ITER)
# print("Done 1")
# model.fit(train_feature_list, train_des_pca)
# print("Done 2")


from sklearn import svm
print("Done 0")
clf = svm.SVC()
print("Done 1")
clf.fit(train_feature_list, train_des_pca)
print("Done 2")

(100, 1001)
(20, 1001)
(100, 2048)
(20, 2048)
Done 0
Done 1


ValueError: bad input shape (100, 1001)

In [ ]:
print("Start 3")
all_predict_test_des = clf.predict(test_feature_list)
print("Done 3")

In [ ]:
add_tag_to_test_des()

In [ ]:
import sklearn.metrics

# return a score
def compare_feature(target, given):
    return float(sklearn.metrics.pairwise.cosine_distances([np.array(target, dtype=float)], [np.array(given, dtype=float)]))
#     return float(np.linalg.norm(np.array(target, dtype=float) - np.array(given, dtype=float)))

def get_top_20(target):
    ret = []
    for index in range(MAX_OUTPUT):
        given = all_predict_test_des[index]
        score = compare_feature(target, given)
        item = str(score) + "___" + str(index)
        ret.append(item)
    return sorted(ret, key=functools.cmp_to_key(cmp))[:20]

# f = open("reverse_count_"+str(count_max_weight)+"unique"+str(uniqueness_max_weight)+".csv", "w")
f = open("reverse_nocount_tag_svm.csv", "w")
f.write("Descritpion_ID,Top_20_Image_IDs\n")
for index in range(MAX_OUTPUT):
    target_desc = test_des_pca[index]
    twenty_image_list = get_top_20(target_desc)
    string_to_write = "" + str(index) + ".txt,"
    for name in twenty_image_list:
        file_name = name.split("___")[1] + ".jpg"+" "
        string_to_write += file_name
    string_to_write = string_to_write.strip() + "\n"
    print(string_to_write)
    f.write(string_to_write)

f.close()